In [ ]:
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from ipywidgets import HBox, VBox

display(HTML("<link rel='stylesheet' type='text/css' href='custom.css'>"))

# Load dataset
df = pd.read_csv("povarenok_recipes_2021_06_16.csv")
df = df.head(100)

def extract_ingredients(ingredient_string):
    matches = re.findall(r"'([^']+)'(?=\s*:\s*)", ingredient_string)
    return matches

df['ingredients_extracted'] = df['ingredients'].apply(extract_ingredients)
df['ingredients_str'] = df['ingredients_extracted'].apply(lambda x: ' '.join(x))

count_vectorizer = CountVectorizer()
ingredient_matrix = count_vectorizer.fit_transform(df['ingredients_str'])

user_favorites = []

def refresh_recommendations(button=None):
    recommended_recipes = recommend_based_on_favorites(user_favorites)
    with output_recs:
        clear_output(wait=True)
        if recommended_recipes.empty:
            display(HTML("<p>No recommendations yet. Add some favorites!</p>"))
        else:
            display(recommended_recipes.style.set_table_attributes('class="recommend-table"'))

def recommend_based_on_favorites(user_favorites, n_recommendations=15):
    indices = df.index[df['name'].isin(user_favorites)].tolist()
    if not indices:
        return pd.DataFrame()
    user_vectors = ingredient_matrix[indices]
    sim_scores = cosine_similarity(user_vectors, ingredient_matrix).mean(axis=0)
    similar_recipes = pd.Series(sim_scores, index=df.index).sort_values(ascending=False)
    recommendations = similar_recipes[~similar_recipes.index.isin(indices)].head(n_recommendations)
    return df.loc[recommendations.index, ['name', 'url']]

def select_recipes(selected_index):
    selected_dish = df.iloc[selected_index]['name']
    if selected_dish not in user_favorites:
        user_favorites.append(selected_dish)
        update_favorites_display()

def update_favorites_display():
    favorites_output.clear_output()
    with favorites_output:
        for dish in user_favorites:
            display(HTML(f'<div class="favorite-item">{dish}</div>'))

dish_dropdown = widgets.Dropdown(
    options=[(name, idx) for idx, name in enumerate(df['name'])],
    disabled=False,
    layout=widgets.Layout(width='250px')
)

select_button = widgets.Button(description="Add to Favorites", layout=widgets.Layout(width='150px'))
select_button.on_click(lambda b: select_recipes(dish_dropdown.value))

output_recs = widgets.Output(layout=widgets.Layout(width='600px', height='400px', overflow='auto'))
favorites_output = widgets.Output(layout=widgets.Layout(width='250px', height='300px', overflow='auto'))

select_button.on_click(refresh_recommendations)

header = HTML("""
<div class="header">
    <h1>RECSYS</h1>
</div>
""")

left_panel = VBox([
    dish_dropdown,
    select_button,
    favorites_output
], layout=widgets.Layout(width='300px', padding='10px'))

right_panel = VBox([
    output_recs
], layout=widgets.Layout(width='600px', padding='10px'))

main_layout = HBox([left_panel, right_panel], layout=widgets.Layout(align_items='center', justify_content='center'))

display(header)
display(main_layout)
refresh_recommendations(None)